In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *
from train_0809 import Trainer

In [2]:
saved_dir = 'saved/1017_SirajMPRA_seq_celltype5_residual/1016_161034'

config_path = f'{saved_dir}/config.yaml'
config = load_config(config_path)
config['gpu_ids'] = [get_free_gpu_id()]
config['train'] = False
config['save_dir'] = saved_dir
config['load_saved_model'] = True
config['saved_model_path'] = os.path.join(saved_dir, 'checkpoint.pth')


config['device'] = get_free_gpu_id()
trainer = Trainer(config, 0)
trainer.test(trainer.test_loader)
torch.cuda.empty_cache()

2024-10-17 13:25:26,441 - INFO - Start non-distributed training on rank 0, cuda:0.
2024-10-17 13:25:28,757 - INFO - load saved model from saved/1017_SirajMPRA_seq_celltype5_residual/1016_161034/checkpoint.pth


100%|██████████| 138/138 [00:37<00:00,  3.72it/s]


In [6]:
# df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_563k.csv', index_col=0)
# df1
# df1.to_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_563k.csv', index=False)

In [8]:
y_pred = np.load(f'{saved_dir}/test_pred.npy')
print(y_pred.shape)

df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_563k.csv')
# df2 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/MPRA_exp/pretrained_based_models/data/Sei_Siraj_features_concat.csv')

# 下面二选一

cell_types = ['HepG2', 'K562', 'SK-N-SH', 'A549', 'HCT116']
cols = [f'{cell_type}_pred' for cell_type in cell_types]
df1[cols] = y_pred
df1

(562654, 5)


,description,seq,A549,HepG2,K562,SK-N-SH,HCT116,is_ref,chr,pos,ref,alt,HepG2_pred,K562_pred,SK-N-SH_pred,A549_pred,HCT116_pred
0,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,0.682885,0.542615,0.549771,0.142351,0.203730
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,0.603323,0.502539,0.455701,0.104369,0.164601
2,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,0.108004,-0.520017,0.210114,0.049328,-0.039143
3,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,-0.006260,-0.580231,0.041053,-0.005394,-0.106598
4,chr1:64764:C:T,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,-0.075728,-0.043544,0.039387,-0.082535,-0.062350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562649,chrX:155233098:T:C,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,-0.010228,0.032687,-0.424491,-0.126130,0.029699
562650,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G,0.390170,0.120498,1.006437,0.442707,0.520425
562651,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G,0.300140,0.073871,0.959080,0.374484,0.434679
562652,chrX:155234730:T:C,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C,0.475473,0.386807,0.036881,0.092868,0.254163


In [33]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(df1), dtype=bool)
data_split_dict['chrom'] = df1['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = df1['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = df1['chr'].isin(['chr1'])
data_split_dict['test'] = df1['chr'].isin(['chr2'])
data_split_dict['designed'] = df1['chr'].isin(['chr0'])


stds = df1[cell_types].std(axis=1, skipna=True)
threshold = np.percentile(stds.dropna(), 90)
data_split_dict['diff'] = (stds > threshold)
data_split_dict['const'] = ~data_split_dict['diff']

# df1['diff'] = (df1[cell_types].max(axis=1) - df1[cell_types].min(axis=1)) > 1
# data_split_dict['const'] = ~df1['diff']
# data_split_dict['diff']  = df1['diff']
print(len(df1), data_split_dict['const'].sum(), data_split_dict['diff'].sum())


keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}_{k2}'] = data_split_dict[k1] & data_split_dict[k2]

562654 506509 56145


In [34]:
for data_split in ['total', 'test', 'const', 'diff', 'test_diff']:
    r_array = pd.DataFrame(index=cell_types, columns=cols)
    for cell_type in cell_types:
        for cell_type_pred in cell_types:
            condition = data_split_dict[data_split]
            df = df1[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
            r_array.loc[cell_type, f'{cell_type_pred}_pred'] = r
            # print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, pearson r = {r:.3f}')
    print(data_split)
    print(r_array)

total
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2     0.928328  0.795064     0.844734  0.800639    0.854892
K562      0.812168  0.931888     0.777655   0.80245    0.825655
SK-N-SH   0.837734   0.75295     0.921466  0.832703     0.81151
A549      0.774922  0.739589     0.808709  0.887499    0.776668
HCT116    0.866934  0.826297     0.832678  0.827199    0.928907
test
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2     0.837403  0.715076      0.75678  0.712307    0.762381
K562      0.740742  0.842592     0.706999   0.73259    0.745613
SK-N-SH   0.750749  0.678025     0.831436  0.746691    0.713272
A549      0.691079  0.668345     0.715399  0.799841    0.680003
HCT116    0.794149  0.749448     0.749991  0.751054    0.847038
const
        HepG2_pred K562_pred SK-N-SH_pred A549_pred HCT116_pred
HepG2     0.908239  0.808063     0.835908  0.794594    0.838676
K562      0.820392  0.899492     0.770749  0.801342    0.810551
SK-N-SH   0.824646  0.7

In [23]:
y_pred_4_0 = np.load(f'saved/0812_SirajMPRA_seq_celltype4/0814_053219/test_pred.npy')
print(y_pred_4_0.shape)

y_pred_4_f = np.load(f'saved/0812_SirajMPRA_seq_feature_celltype4/0815_041254/test_pred.npy')
print(y_pred_4_f.shape)

y_pred_3_H = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_041826/test_pred.npy')
print(y_pred_3_H.shape)

y_pred_3_A = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0819_005952/test_pred.npy')
print(y_pred_3_A.shape)

y_pred_3_K = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_214824/test_pred.npy')
print(y_pred_3_K.shape)

y_pred_3_H = np.load(f'saved/0815_SirajMPRA_seq_feature_celltype3/0815_214850/test_pred.npy')
print(y_pred_3_H.shape)

(585605, 4)
(1765218,)
(1765218,)
(1765218,)
(1765218,)
(1765218,)


In [25]:
df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
# print(df1.head())

df2 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/MPRA_exp/pretrained_based_models/data/Sei_Siraj_features_concat.csv')
# print(df2.head())


# 下面二选一

for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    df1[f'{cell_type}_pred'] = y_pred_4_0[:, i]
df1


# df2['pred'] = y_pred_4_0
# for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
#     df = df2[df2['cell_type'] == cell_type][['seq', 'pred']]
#     df.rename(columns={'pred': f'{cell_type}_pred'}, inplace=True)
#     df = df.drop_duplicates(subset=['seq'])
#     df1 = pd.merge(df1, df, on='seq', how='left')
# df1

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,0.860727,0.471197,0.197887,0.369649
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,0.745834,0.418501,0.144777,0.290741
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,0.044020,-0.045745,-0.043967,-0.115304
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,0.009691,-0.069309,-0.071219,-0.152117
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,-0.129289,-0.136357,-0.108981,-0.104691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585600,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,-0.130473,0.371278,-0.105545,-0.168190
585601,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G,1.410156,0.740903,0.845696,0.935793
585602,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G,1.131462,0.488258,0.668862,0.834238
585603,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C,0.393603,0.419445,0.133474,0.314708


In [26]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(df1), dtype=bool)
data_split_dict['chrom'] = df1['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = df1['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = df1['chr'].isin(['chr1'])
data_split_dict['test'] = df1['chr'].isin(['chr2'])
data_split_dict['designed'] = df1['chr'].isin(['chr0'])


cell_types = ['HepG2', 'K562', 'A549', 'HCT116']
df1['diff'] = (df1[cell_types].max(axis=1) - df1[cell_types].min(axis=1)) > 1
data_split_dict['const'] = ~df1['diff']
data_split_dict['diff']  = df1['diff']
print(len(df1), data_split_dict['const'].sum(), data_split_dict['diff'].sum())


# df1['mean'] = df1[cell_types].mean(axis=1)
# df1['std'] = df1[cell_types].std(axis=1)

# threshold = df1['std'].quantile(0.2)
# data_split_dict['diff'] = (df1['std'] >= threshold)
# data_split_dict['const'] = (df1['std'] < threshold)

# print(data_split_dict['diff'].sum(), data_split_dict['const'].sum())

# # for cell_type in cell_types:
# #     # df1[f'{cell_type}_zscore'] = (df1[cell_type] - df1['mean']) / df1['std']
# #     # df1[f'{cell_type}_diff'] = df1[f'{cell_type}_zscore'].abs() > 1
# #     df1[f'{cell_type}_diff'] = (df1[cell_type] - df1['mean']).abs() > 1
# #     df1[f'{cell_type}_const'] = ~df1[f'{cell_type}_diff']
# #     print(f"{cell_type}_diff num = {df1[f'{cell_type}_diff'].sum()}")

In [28]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test', 'const', 'diff']:
            condition = data_split_dict[data_split]
            df = df1[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}'])
            print(f'{cell_type:6} true vs {cell_type_pred:6} true, {data_split:5}, pearson r = {r:.3f}')
        print()

HepG2  true vs HepG2  true, total, pearson r = 1.000
HepG2  true vs HepG2  true, train, pearson r = 1.000


HepG2  true vs HepG2  true, valid, pearson r = 1.000
HepG2  true vs HepG2  true, test , pearson r = 1.000
HepG2  true vs HepG2  true, const, pearson r = 1.000
HepG2  true vs HepG2  true, diff , pearson r = 1.000

HepG2  true vs K562   true, total, pearson r = 0.787
HepG2  true vs K562   true, train, pearson r = 0.789
HepG2  true vs K562   true, valid, pearson r = 0.779
HepG2  true vs K562   true, test , pearson r = 0.770
HepG2  true vs K562   true, const, pearson r = 0.833
HepG2  true vs K562   true, diff , pearson r = 0.651

HepG2  true vs A549   true, total, pearson r = 0.738
HepG2  true vs A549   true, train, pearson r = 0.740
HepG2  true vs A549   true, valid, pearson r = 0.737
HepG2  true vs A549   true, test , pearson r = 0.724
HepG2  true vs A549   true, const, pearson r = 0.705
HepG2  true vs A549   true, diff , pearson r = 0.656

HepG2  true vs HCT116 true, total, pearson r = 0.867
HepG2  true vs HCT116 true, train, pearson r = 0.869
HepG2  true vs HCT116 true, valid, pearson 

In [29]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in [cell_type]:
        for data_split in ['total', 'train', 'valid', 'test', 'const', 'diff']:
            condition = data_split_dict[data_split]
            df = df1[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, pearson r = {r:.3f}')
        print()

HepG2 true vs HepG2 pred, total, pearson r = 0.895
HepG2 true vs HepG2 pred, train, pearson r = 0.911
HepG2 true vs HepG2 pred, valid, pearson r = 0.813
HepG2 true vs HepG2 pred, test, pearson r = 0.811
HepG2 true vs HepG2 pred, const, pearson r = 0.858
HepG2 true vs HepG2 pred, diff, pearson r = 0.881

K562 true vs K562 pred, total, pearson r = 0.905
K562 true vs K562 pred, train, pearson r = 0.922
K562 true vs K562 pred, valid, pearson r = 0.823
K562 true vs K562 pred, test, pearson r = 0.816
K562 true vs K562 pred, const, pearson r = 0.849
K562 true vs K562 pred, diff, pearson r = 0.898

A549 true vs A549 pred, total, pearson r = 0.847
A549 true vs A549 pred, train, pearson r = 0.864
A549 true vs A549 pred, valid, pearson r = 0.767
A549 true vs A549 pred, test, pearson r = 0.760
A549 true vs A549 pred, const, pearson r = 0.728
A549 true vs A549 pred, diff, pearson r = 0.856

HCT116 true vs HCT116 pred, total, pearson r = 0.901
HCT116 true vs HCT116 pred, train, pearson r = 0.918
HCT

In [30]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in [cell_type]:
        for data_split in ['test']:
            for data_split_2 in ['total', 'const', 'diff']:
                condition = data_split_dict[data_split] & data_split_dict[data_split_2] #df1[f'{cell_type}_{data_split_2}']
                df = df1[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type:6} true vs {cell_type_pred:6} pred, {data_split:5}, {data_split_2:4}, pearson r = {r:.3f}')
        print()

HepG2  true vs HepG2  pred, test , total, pearson r = 0.811
HepG2  true vs HepG2  pred, test , const, pearson r = 0.768
HepG2  true vs HepG2  pred, test , diff, pearson r = 0.783

K562   true vs K562   pred, test , total, pearson r = 0.816
K562   true vs K562   pred, test , const, pearson r = 0.749
K562   true vs K562   pred, test , diff, pearson r = 0.791

A549   true vs A549   pred, test , total, pearson r = 0.760
A549   true vs A549   pred, test , const, pearson r = 0.612
A549   true vs A549   pred, test , diff, pearson r = 0.762

HCT116 true vs HCT116 pred, test , total, pearson r = 0.822
HCT116 true vs HCT116 pred, test , const, pearson r = 0.804
HCT116 true vs HCT116 pred, test , diff, pearson r = 0.810



In [31]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test']:
            for data_split_2 in ['total', 'const', 'diff']:
                condition = data_split_dict[data_split] & data_split_dict[data_split_2]
                df = df1[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, {data_split_2}, pearson r = {r:.3f}')
            print()

HepG2 true vs HepG2 pred, total, total, pearson r = 0.895
HepG2 true vs HepG2 pred, total, const, pearson r = 0.858
HepG2 true vs HepG2 pred, total, diff, pearson r = 0.881

HepG2 true vs HepG2 pred, train, total, pearson r = 0.911
HepG2 true vs HepG2 pred, train, const, pearson r = 0.876
HepG2 true vs HepG2 pred, train, diff, pearson r = 0.902

HepG2 true vs HepG2 pred, valid, total, pearson r = 0.813
HepG2 true vs HepG2 pred, valid, const, pearson r = 0.770
HepG2 true vs HepG2 pred, valid, diff, pearson r = 0.785

HepG2 true vs HepG2 pred, test, total, pearson r = 0.811
HepG2 true vs HepG2 pred, test, const, pearson r = 0.768
HepG2 true vs HepG2 pred, test, diff, pearson r = 0.783

HepG2 true vs K562 pred, total, total, pearson r = 0.790
HepG2 true vs K562 pred, total, const, pearson r = 0.791
HepG2 true vs K562 pred, total, diff, pearson r = 0.697

HepG2 true vs K562 pred, train, total, pearson r = 0.806
HepG2 true vs K562 pred, train, const, pearson r = 0.810
HepG2 true vs K562 pre

In [18]:
# saved_dir = 'saved/0625_SirajMPRA_A549/0625_025532'

HepG2_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0613_231345/y_pred_list.npy')
K562_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0618_001655/y_pred_list.npy')
A549_pred = np.load(f'saved/0625_SirajMPRA_A549/0625_025532/y_pred_list.npy')
HCT116_pred = np.load(f'saved/0625_SirajMPRA_HCT116/0625_025546/y_pred_list.npy')
y_pred_list = np.concatenate([HepG2_pred, K562_pred, A549_pred, HCT116_pred], axis=1)
# print(y_pred_list.shape)

df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')
for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    df1[f'{cell_type}_pred'] = y_pred_list[:, i]
df1

/tmp/ipykernel_694925/4012534825.py:10: DtypeWarning: Columns (6,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')


,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,origin,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677.0,G,A,NaN,0.603010,0.979300,0.089610,0.228658
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677.0,G,A,NaN,0.523534,0.916531,0.029055,0.214567
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697.0,T,C,NaN,-0.025966,-0.583377,0.334277,-0.240863
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697.0,T,C,NaN,-0.049796,-0.607013,0.293812,-0.277427
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764.0,C,T,NaN,-0.114398,-0.093055,-0.052438,-0.148054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661768,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,NaN,0.591628,-0.315267,0.068697,NaN,NaN,chr0,NaN,NaN,NaN,control,1.488066,-0.080938,0.131021,-0.371440
661769,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,NaN,-0.359782,-0.134821,-0.022281,NaN,NaN,chr0,NaN,NaN,NaN,control,-0.775348,-0.264292,0.012704,-0.667063
661770,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,NaN,0.489900,0.646673,0.462366,NaN,NaN,chr0,NaN,NaN,NaN,control,0.356786,0.398112,0.177435,0.297358
661771,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,NaN,-0.014411,-0.061676,0.239693,NaN,NaN,chr0,NaN,NaN,NaN,control,1.167894,0.079996,0.012424,0.225529


In [21]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            r = pearson(df1[condition][f'{cell_type}'], df1[condition][f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
            print('pearson r', r)
        print()

HepG2 true vs HepG2 pred, train data, pearson r = 0.926
pearson r 0.9259957006812582
HepG2 true vs HepG2 pred, valid data, pearson r = 0.817
pearson r 0.8171515482957136
HepG2 true vs HepG2 pred, test data, pearson r = 0.815
pearson r 0.8148071042476853
HepG2 true vs HepG2 pred, designed data, pearson r = 0.745
pearson r 0.744512519022009

HepG2 true vs K562 pred, train data, pearson r = 0.759
pearson r 0.7587345782678894
HepG2 true vs K562 pred, valid data, pearson r = 0.690
pearson r 0.6903916498834725
HepG2 true vs K562 pred, test data, pearson r = 0.685
pearson r 0.6847371073960745
HepG2 true vs K562 pred, designed data, pearson r = -0.296
pearson r -0.2956059602354766

HepG2 true vs A549 pred, train data, pearson r = 0.632
pearson r 0.6322560229429839
HepG2 true vs A549 pred, valid data, pearson r = 0.583
pearson r 0.582653973730981
HepG2 true vs A549 pred, test data, pearson r = 0.574
pearson r 0.573519726677123
HepG2 true vs A549 pred, designed data, pearson r = 0.033
pearson r 

In [52]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = df1[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}' for ct in cell_types if ct!= cell_type]
            y = df1[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562', 'A549', 'HCT116'] pred, train data, pearson r = 0.852
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, valid data, pearson r = 0.843
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, test data, pearson r = 0.836
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

K562 true vs ['HepG2', 'A549', 'HCT116'] pred, train data, pearson r = 0.819
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, valid data, pearson r = 0.815
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, test data, pearson r = 0.804
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

A549 true vs ['HepG2', 'K562', 'HCT116'] pred, train data, pearson r = 0.779
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, valid data, pearson r = 0.782
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, test data, pearson r = 0.762
0 (less than 10%) values are non-nan.
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, designed data, pearson r = nan

HCT116 true vs ['HepG2', 'K56

In [24]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = df1[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}_pred' for ct in cell_types if ct!= cell_type]
            y = df1[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.810
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.727
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.723
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.133

K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.811
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.745
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.740
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.147

A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, train data, pearson r = 0.762
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.708
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, test data,

In [60]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((df1['HepG2'] - df1['K562']).abs() > 2)
            x = df1[condition][f'{cell_type}']
            y = df1[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((df1['HepG2'] - df1['K562']).abs() <= 2)
            x = df1[condition][f'{cell_type}']
            y = df1[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


880
HepG2 true vs HepG2 pred, test data, pearson r = 0.670

880
HepG2 true vs K562 pred, test data, pearson r = 0.044

880
HepG2 true vs A549 pred, test data, pearson r = 0.166

880
HepG2 true vs HCT116 pred, test data, pearson r = 0.258

880
K562 true vs HepG2 pred, test data, pearson r = 0.133

880
K562 true vs K562 pred, test data, pearson r = 0.695

880
K562 true vs A549 pred, test data, pearson r = 0.386

880
K562 true vs HCT116 pred, test data, pearson r = 0.447

880
A549 true vs HepG2 pred, test data, pearson r = 0.484

880
A549 true vs K562 pred, test data, pearson r = 0.390

880
A549 true vs A549 pred, test data, pearson r = 0.585

880
A549 true vs HCT116 pred, test data, pearson r = 0.524

880
HCT116 true vs HepG2 pred, test data, pearson r = 0.549

880
HCT116 true vs K562 pred, test data, pearson r = 0.445

880
HCT116 true vs A549 pred, test data, pearson r = 0.578

880
HCT116 true vs HCT116 pred, test data, pearson r = 0.654

43603
HepG2 true vs HepG2 pred, test data, pears